In [1]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import datetime
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import json
init_notebook_mode(connected=True)
import pandas as pd

In [2]:
def create_plot():

    df = pd.read_csv('./stat_permonth.csv')
#     iplot([{"x": df['monthonly'], "y": df['counts']}])
    data = [go.Scatter(x=df['monthonly'] ,y=df['counts'])]
    graphJSON = json.dumps(data, cls=plotly.utils.PlotlyJSONEncoder)
    df = pd.read_csv('./stat_perday.csv')
    data = [go.Scatter(x=df['dateonly'] ,y=df['counts'])]
    graphJSON2 = json.dumps(data, cls=plotly.utils.PlotlyJSONEncoder)

    return graphJSON,graphJSON2

In [122]:
from flask import Flask, render_template, request


app = Flask(__name__)


@app.route("/graph")
def graph():

    gr1,gr2 = create_plot()
    return render_template('graph.html',plot=gr1,plot2=gr2)

@app.route("/")
def index():
    
    return render_template("index.html")

@app.route("/index")
def index2():
    
    return render_template("index.html")

@app.route("/keyword")
def keyword():
    
    return render_template("keyword.html")

@app.route("/hi",methods=['GET','POST'])
def hi():

    return render_template("submit.html")

@app.route("/select")
def select():

    return render_template("select.html")

@app.route("/dash")
def dash():

    return render_template("dashboard.html")


@app.route("/test",methods=['GET','POST'])
def test():
    if request.method == 'POST':
        result = request.form
        start = result['from']
        end = result['to']
        num = result['topic_num']
        new_result=dict()
        new_result['from'] = str(start)+"nnnn"
        new_result['end'] = str(end)+"kkkk"
        new_result['num'] = num*1000
    elif request.method=="GET":
        
        new_result=dict()
        start = request.args.get('from')
        end = request.args.get('to')
        num = request.args.get('topic_num')
        
        
        new_result['from'] = str(start)+"nnnn"
        new_result['end'] = str(end)+"kkkk"
        new_result['num'] = num*1000
    
    return render_template("test.html",result = new_result)


if __name__ == "__main__":
    app.run(host='59.29.224.81',threaded=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://59.29.224.81:5000/ (Press CTRL+C to quit)
59.29.224.81 - - [26/Mar/2019 14:56:20] "GET / HTTP/1.1" 200 -
59.29.224.81 - - [26/Mar/2019 14:56:23] "GET /graph HTTP/1.1" 200 -
59.29.224.81 - - [26/Mar/2019 14:56:25] "GET /graph2 HTTP/1.1" 200 -
59.29.224.81 - - [26/Mar/2019 14:56:54] "GET / HTTP/1.1" 200 -
59.29.224.81 - - [26/Mar/2019 14:56:54] "GET /static/js/side_menu.js HTTP/1.1" 200 -
59.29.224.81 - - [26/Mar/2019 14:56:54] "GET /static/css/theme2.css HTTP/1.1" 200 -
59.29.224.81 - - [26/Mar/2019 14:56:54] "GET /static/css/theme.css HTTP/1.1" 200 -
59.29.224.81 - - [26/Mar/2019 14:56:54] "GET /static/img/hynix.png HTTP/1.1" 200 -
59.29.224.81 - - [26/Mar/2019 14:56:54] "GET /static/img/title2.png HTTP/1.1" 200 -
59.29.224.81 - - [26/Mar/2019 14:56:54] "GET /static/img/graph.png HTTP/1.1" 200 -
59.29.224.81 - - [26/Mar/2019 14:56:54] "GET /static/img/keyword.png HTTP/1.1" 200 -
59.29.224.81 - - [26/Mar/2019 14:56:59] "GET /select HTTP/1.1" 200 -
59.29.224.81 - - [

In [37]:
d1 = pd.read_csv('./crawling.csv')

In [63]:
d2 = d1[d1['voc_type'].isin(li)]

In [59]:
li = set(d1['voc_type'])

In [60]:
li = list(li)[1:]

In [65]:
d3 = d2.groupby(['voc_type']).size().reset_index(name='counts')

In [69]:
labels = list(d3['voc_type'])
values = list(d3['counts'])

In [75]:
import plotly.plotly as py
import plotly.graph_objs as go
# import datetime
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import datetime

In [114]:
tra = go.Pie(labels=labels[:5], values=values[:5], hole=.7, name="Hi")
lay = go.Layout(font={"family":"뫼비우스 Regular"},showlegend=False,annotations=[{"text":"버스","x":0.5,"y":0.5,"showarrow":False,"font":{"family":"뫼비우스 Regular","size":50,"color":"black"}}])
fig = go.Figure(data=[tra], layout=lay)
iplot(fig)


In [115]:
plot3 =  json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)